In [2]:
import sys
import os
import pandas as pd
import time
from datetime import datetime #make sure this notebook is in the same directory as the conway_lib folder
from conway_lib import ConwayGame
conway_game=ConwayGame()


def generate_sets(A=100, N=30, I=2, Toroidal=False, save_folder='C:\\Users\\jaime\\ML_Playground_1\\'):
    # Create a timestamp for the current date and time
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    # Ensure that the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Create an instance of ConwayGame
    game = ConwayGame(width=N, height=N, grid_size=1)

    # Start timing the process
    start_time = time.time()
    
    # Generate the validation sets using the method provided
    start = 0 #order param of the frist sample
    end = 1 #order param of the last sample
    train_data = game.generate_sets(A=A, N=N, I=I, s=start, e=end)

    # Process and save the data to CSV
    data = []
    for index in range(A):
        states = train_data[index]
        data.append(states)
        
    # Save the flattened states to CSV
    columns = [f'State {i + 1}' for i in range(I)]
    df = pd.DataFrame(data, columns=columns)
    print(df.shape)
    
    toroidal_str = 'toroidal' if Toroidal else 'non_toroidal'
    save_path = os.path.join(save_folder, f'EXAMPLE_conway_states_{start}_{end}_{A}by{N}by{N}by{I}_{toroidal_str}_{timestamp}.csv')
    df.to_csv(save_path, index=False)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"CSV generation time: {elapsed_time} seconds")

# Example usage
generate_sets(A=100, N=50, I=20, Toroidal=True, save_folder='Conway_GPT_v8\\')

(100, 20)
CSV generation time: 31.33033275604248 seconds


## Testing Set Generator:

In [ ]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from conway_lib.game import ConwayGame

def generate_test_sets(N=32, I=10, Toroidal=True, save_folder='C:\\Users\\jaime\\ML_Playground_1\\Game_Testing'):
    # Create a timestamp for the current date and time
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    # Ensure that the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Create an instance of ConwayGame
    game = ConwayGame(toroidal=Toroidal, width=N, height=N, grid_size=1)

    # Start timing the process
    start_time = time.time()

    # Create the test set
    test_data = []
    order_params = [0, 0.25, 0.5, 0.75, 1]
    for op in order_params:
        game.randomize_grid_uniform(op)
        game.run(num_iterations=I)

        metagrid = game.metagrid
        states = [game.get_state_as_string(metagrid[i]) for i in range(I)]
        test_data.append(states)
        animate_game(game, I, f'order_param_{op}.gif')

    specific_patterns = ["gliders", "cloverleaf", "hammerhead_spaceship", "blinkers", "r_pentomino"]
    for pattern in specific_patterns:
        game.initialize_pattern(pattern)
        game.run(num_iterations=I)
        metagrid = game.metagrid
        states = [game.get_state_as_string(metagrid[i]) for i in range(I)]
        test_data.append(states)
        animate_game(game, I, f'{pattern}.gif')

    # Save the test data to CSV
    columns = [f'State {i + 1}' for i in range(I)]
    df = pd.DataFrame(test_data, columns=columns)
    print(df.shape)
    
    save_path = os.path.join(save_folder, f'conway_test_states_{N}by{N}_{timestamp}.csv')
    df.to_csv(save_path, index=False)
    print(f"Test set saved to: {save_path}")

    # End timing the process
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"CSV generation time: {elapsed_time} seconds")

def animate_game(game, num_iterations, filename, interval=200):
    fig, ax = plt.subplots()

    def update(frame):
        ax.clear()
        ax.imshow(game.metagrid[frame], cmap='binary')
        ax.set_title(f"Iteration {frame+1}")

    ani = FuncAnimation(fig, update, frames=num_iterations, interval=interval, repeat=False)
    
    # Save the animation as a GIF file
    writer = PillowWriter(fps=5)
    ani.save(filename, writer=writer)
    plt.close()

# Example usage
generate_test_sets(N=32, I=10, Toroidal=True, save_folder='C:\\Users\\jaime\\ML_Playground_1\\Conway_GPT_v8\\')


## Testing Set Generator
The following is code assuming we want to use the same ICs are we previously generated using the afformentioned testing set generator script, but this time we calculate 249 iterations instead of 9. 
 

In [26]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from conway_lib.game import ConwayGame

def generate_test_sets_2(N=32, I=10, Toroidal=True, save_folder='C:\\Users\\jaime\\ML_Playground_1\\Game_Testing'):
    # Create a timestamp for the current date and time
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    # Ensure that the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Create an instance of ConwayGame
    game = ConwayGame(toroidal=Toroidal, width=N, height=N, grid_size=1)

    # Start timing the process
    start_time = time.time()

    # Read initial states from the CSV file
    initial_states_csv = 'Conway_GPT_v8\\conway_test_states_32by32_20240716_151502.csv'
    df_initial_states = pd.read_csv(initial_states_csv)
    initial_states = df_initial_states.iloc[:, 0].tolist()  # Extract the first column

    # Create the test set
    test_data = []

    # Use initial states from the CSV for order parameter samples
    for state_string in initial_states:
        game.set_grid_from_string(state_string)
        game.run(num_iterations=I)

        metagrid = game.metagrid
        states = [game.get_state_as_string(metagrid[i]) for i in range(I)]
        test_data.append(states)
        # animate_game(game, I, f'order_param_{state_string[:10]}.gif')  # Use a portion of the state as a filename

    # specific_patterns = ["gliders", "cloverleaf", "hammerhead_spaceship", "blinkers", "r_pentomino"]
    # for pattern in specific_patterns:
    #     game.initialize_pattern(pattern)
    #     game.run(num_iterations=I)
    #     metagrid = game.metagrid
    #     states = [game.get_state_as_string(metagrid[i]) for i in range(I)]
    #     test_data.append(states)
    #     # animate_game(game, I, f'{pattern}.gif')

    # Save the test data to CSV
    columns = [f'State {i + 1}' for i in range(I)]
    df = pd.DataFrame(test_data, columns=columns)
    print(df.shape)
    
    save_path = os.path.join(save_folder, f'conway_test_states_{N}by{N}_{timestamp}.csv')
    df.to_csv(save_path, index=False)
    print(f"Test set saved to: {save_path}")

    # End timing the process
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"CSV generation time: {elapsed_time} seconds")

# def animate_game(game, num_iterations, filename, interval=200):
#     fig, ax = plt.subplots()

#     def update(frame):
#         ax.clear()
#         ax.imshow(game.metagrid[frame], cmap='binary')
#         ax.set_title(f"Iteration {frame+1}")

#     ani = FuncAnimation(fig, update, frames=num_iterations, interval=interval, repeat=False)
    
#     # Save the animation as a GIF file
#     writer = PillowWriter(fps=5)
#     ani.save(filename, writer=writer)
#     plt.close()

# Example usage
generate_test_sets_2(N=32, I=250, Toroidal=True, save_folder='Conway_GPT_v8\\')


(10, 250)
Test set saved to: Conway_GPT_v8\conway_test_states_32by32_20240827_172807.csv
CSV generation time: 17.484988689422607 seconds


In [28]:
import pandas as pd

# Define the paths to the old and new CSV files
old_csv_path = 'Conway_GPT_v8\\conway_test_states_32by32_20240716_151502.csv'
new_csv_path = 'Conway_GPT_v8\\conway_test_states_32by32_20240827_172807.csv'

# Read the first 10 rows of data (excluding headers) from each CSV file
# Adjust the number of columns to match the minimum number present in both files
old_df = pd.read_csv(old_csv_path, nrows=10)
new_df = pd.read_csv(new_csv_path, nrows=10)

# Find the minimum number of columns in both DataFrames
min_columns = min(len(old_df.columns), len(new_df.columns))

# Subset both DataFrames to have the same number of columns
old_df = old_df.iloc[:, :min_columns]
new_df = new_df.iloc[:, :min_columns]

# Align columns by renaming to ensure they match
old_df.columns = new_df.columns

# Compare the first 10 rows of data (ignoring the headers)
are_identical = old_df.equals(new_df)

# Output the result
if are_identical:
    print("The first 10 states of the old and new test CSV files are identical.")
else:
    print("The first 10 states of the old and new test CSV files are different.")

# Optional: Print the differences
if not are_identical:
    print("\nDifferences between old and new CSVs (first 10 states):")
    comparison = old_df.compare(new_df, align_axis=1)
    print(comparison)


The first 10 states of the old and new test CSV files are identical.


In [25]:
import pandas as pd
import os
import numpy as np
from conway_lib.game import ConwayGame
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# Define the path to the new CSV file
csv_path = 'Conway_GPT_v8\\conway_test_states_32by32_20240827_132431.csv'

# Sample names in order
sample_names = [
    "Order_Param_0",
    "Order_Param_0.25",
    "Order_Param_0.5",
    "Order_Param_0.75",
    "Order_Param_1",
    "Glider",
    "Cloverleaf",
    "Hammerhead_Spaceship",
    "Blinkers",
    "r-Pentomino"
]

# Read the CSV file
df = pd.read_csv(csv_path)

# Directory to save the animations
save_folder = 'Testing_Animations//Ground_Truth_250_Iterations'
os.makedirs(save_folder, exist_ok=True)

# Determine the number of iterations based on the number of columns
num_iterations = len(df.columns)

# Function to animate the game
def animate_game(game, num_iterations, filename, interval=200):
    fig, ax = plt.subplots()

    def update(frame):
        ax.clear()
        ax.imshow(game.metagrid[frame], cmap='binary')
        ax.set_title(f"Iteration {frame + 1}")

    ani = FuncAnimation(fig, update, frames=num_iterations, interval=interval, repeat=False)
    
    # Save the animation as a GIF file
    writer = PillowWriter(fps=10)
    ani.save(filename, writer=writer)
    plt.close()

# Loop through each row in the DataFrame and create animations
for index, row in df.iterrows():
    initial_state_string = row['State 1']  # Assume the initial state is in 'State 1' column

    # Create a new instance of ConwayGame
    game = ConwayGame(width=32, height=32, grid_size=1, toroidal=True)
    
    # Set the grid from the string
    game.set_grid_from_string(initial_state_string)
    
    # Run the simulation
    game.run(num_iterations=num_iterations)
    
    # Save the animation
    animation_filename = os.path.join(save_folder, f"{sample_names[index]}.gif")
    animate_game(game, num_iterations, animation_filename)

    print(f"Animation saved as {animation_filename}")


Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Order_Param_0.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Order_Param_0.25.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Order_Param_0.5.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Order_Param_0.75.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Order_Param_1.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Glider.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Cloverleaf.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Hammerhead_Spaceship.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\Blinkers.gif
Animation saved as Testing_Animations//Ground_Truth_250_Iterations\r-Pentomino.gif


IndexError: list index out of range